<a href="https://colab.research.google.com/github/danort92/Programming-with-Python/blob/main/Python_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import os
from datetime import datetime





def help_cmd():

  """
  Function that prints an help message with infos about the commands to use to run the program
  """
  
  print("\nThe available commands are the following:\n"+
        "add: add aproduct to the stock\n"+
        "list: list the products stocked\n"+
        "sell: register a sale\n"+
        "profits: show the total gross and net profits\n"+
        "help: show the possible commands\n"+
        "close: exit the program")

    





def stock_info():

  """
  Function that generates the timely stock situation, starting from the .csv file where purchased and sold products are registered.

  1-stock.csv, created with the first purchased product, is read. Each row (corresponding to a purchased or sold product) is registered in a list of dictionaries
  2-the list of dictionaries is modified: for each product, the presence of the same one in the previous dictionaries gets checked. 
    If it exists its quantity is added to the previous dictionary.
  3-just the first occurence for each product contains now the right total quantity, the other rows are cancelled: the final list of dictionaries corresponds to the stock
  4-the functions returns tmp: the key is the product, its values is a dictionary with the folloing infos: quantity, purchase price and selling price 
  """
  
  with open("stock.csv","r") as csv_file:        
        csv_reader=csv.DictReader(csv_file)
        list_dict=[]
        for row in list(csv_reader):
            list_dict.append(row)

        for i,line in enumerate(list_dict):
            prev_lines=[]
            for j in range(i):
                prev_lines.append(list_dict[j]["PRODUCT"])
            for k,prev_line in enumerate(prev_lines):
                if line["PRODUCT"]==prev_line:
                    list_dict[k]["QUANTITY"]=int(list_dict[k]["QUANTITY"])+int(line["QUANTITY"])
                else:
                    continue

        tmp={}           
        for i,row in enumerate(list_dict):
            prev_lines=[]
            for j in range(i):
                prev_lines.append(list_dict[j]["PRODUCT"])
    
            if not row["PRODUCT"] in prev_lines:

                info={}
                info["Quantity"]=int(row["QUANTITY"])
                info["Purchase price"]=float(row["PURCHASE PRICE"])
                info["Selling price"]=float(row["SELLING PRICE"])
                tmp[row["PRODUCT"]]=info
            else:
                continue
        return tmp 
    
    






def add(product_name,quantity):
    
    """
    Function that adds the purchased products to the stock. It receives variables product_name and quantity as inputs.

    1-stock.csv file (created with the first purchased product) is opened in append mode (new rows are added at the end of the file)
    2-the emptiness of the file is checked: if it's empty the header with infos about product name, quantity, purchased price, selling price, operation type (buy, sale) and operation date-time
    3-a list with product names is created and the presence of the inserted product is checked: 
       I) if it does not exist the infos about purchase price and selling price are asked and a new row is written in the .csv file
      II) if the product already exists a new row is created with purchase and selling prices already registered for that product

    """

    with open("stock.csv","a+",newline="") as csv_file:
        columns=["PRODUCT","QUANTITY","PURCHASE PRICE","SELLING PRICE","OPERATION","DATE TIME"]
        csv_reader=csv.DictReader(csv_file)
        csv_writer=csv.DictWriter(csv_file,fieldnames=columns)

        
        fileEmpty = os.stat("stock.csv").st_size == 0
        if fileEmpty:
            csv_writer.writeheader()  

        
        list_products=[]
        csv_file.seek(0)
        for row in csv_reader:
            list_products.append(row["PRODUCT"])
            
        
        if not product_name in set(list_products):
            purchase_price,selling_price=None,None
            while purchase_price==None:
                try:
                    purchase_price=float(input("Purchase price: "))
                except ValueError:
                    print("\nPurchase price not valid: insert decimal number")
            while selling_price==None:
                try:        
                    selling_price=float(input("Selling price: "))
                except ValueError:
                    print("\nSelling price not valid: insert decimal number")

            
            csv_writer.writerow({"PRODUCT":product_name,"QUANTITY":quantity,"PURCHASE PRICE":purchase_price,"SELLING PRICE":selling_price,"OPERATION":"buy","DATE TIME":datetime.strftime(datetime.now(), "%d-%m-%Y %H.%M.%S")})

        
        else:
            csv_file.seek(0)
            list_lines=[]
            for row in list(csv_reader)[1:]:
                list_lines.append(row)
            for i,product in enumerate(list_products):
                if product_name==product:
                    z=i
                    break
                else:
                    continue
            
            csv_writer.writerow({"PRODUCT":product_name,"QUANTITY":quantity,"PURCHASE PRICE":list_lines[z]["PURCHASE PRICE"],"SELLING PRICE":list_lines[z]["SELLING PRICE"],"OPERATION":"buy","DATE TIME":datetime.strftime(datetime.now(), "%d-%m-%Y %H.%M.%S")})
        
        print(f"ADDED {quantity} X {product_name}")
    
    stock_info()
        









def itemize():
    
    """
    Function that lists the stock.

    1-the presence of a file named stock.csv is checked. If it does not exist an error is shown
    2-for each product (with quantity >0) in stock the name, quantity and selling price are printed

    """

    try:
        stock=stock_info()
        head_text=["\nPRODUCT","QUANTITY","PRICE"]
        print(*head_text)

        for i in range(len(list(stock.keys()))):
            if list(stock.values())[i]["Quantity"]!=0:
                print(f"{list(stock.keys())[i]} "+
                f"{list(stock.values())[i]['Quantity']} "+
                f"€{list(stock.values())[i]['Selling price']:.2f} ")
    except FileNotFoundError:
        print("The register purchases/sales does not exist. Maybe the first purchase has not been registered yet?...")
        
        
        








def sell(product_name,quantity):
    
    """
    Function that adds the sold products to the stock.

    1-stock.csv file is opened in append mode (new rows are added at the end of the file)
    2-a list with product names is created and the presence of the inserted product is checked:
       I) if it does not exist a "product unavailable" error is shown
      II) if it already exists:
            i) each row of stock.csv (corresponding to a purchased or sold product) is registered in a list of dictionaries
           ii) the list of dictionaries is modified: for each product, the presence of the same one in the previous dictionaries gets checked. 
              If it exists its quantity is added to the previous dictionary
          iii) just the first occurence for each product contains now the right total quantity.
                The other rows are cancelled: the final list of dictionaries corresponds to the stock before the sale
           iv) the availability of product is checked. If the quantity is enough a new row is added to the .csv file, otherwise an error is printed
            v) if sale is registered variable stop=1, otherwise 0, and function stock_info is called to update the stock
    """

    with open("stock.csv","a+",newline="") as csv_file:
        columns=["PRODUCT","QUANTITY","PURCHASE PRICE","SELLING PRICE","OPERATION","DATE TIME"]
        csv_reader=csv.DictReader(csv_file)
        csv_writer=csv.DictWriter(csv_file,fieldnames=columns)
        fileEmpty = os.stat("stock.csv").st_size == 0
        if fileEmpty:
            csv_writer.writeheader()  
        
        list_products=[]
        csv_file.seek(0)
        for row in csv_reader:
            list_products.append(row["PRODUCT"])

        if not product_name in set(list_products):
            print("Product not present in stock")
        
        else:
            csv_file.seek(0)
            csv_reader=csv.DictReader(csv_file)
            list_lines=[]
            for row in list(csv_reader):
                list_lines.append(row)
            for i,product in enumerate(list_products):
                if product_name==product:
                    z=i
                    break
                else:
                    continue
            
            csv_file.seek(0) 
            csv_reader=csv.DictReader(csv_file)
            list_dict=[]
            for row in list(csv_reader):
                list_dict.append(row)
            
            for i,line in enumerate(list_dict):
                prev_lines=[]
                for j in range(i):
                    prev_lines.append(list_dict[j]["PRODUCT"])
                for k,prev_line in enumerate(prev_lines):
                    if line["PRODUCT"]==prev_line:
                        list_dict[k]["QUANTITY"]=int(list_dict[k]["QUANTITY"])+int(line["QUANTITY"])
                    else:
                        continue

            tmp={}           
            for i,row in enumerate(list_dict):
                prev_lines=[]
                for j in range(i):
                    prev_lines.append(list_dict[j]["PRODUCT"])

                if not row["PRODUCT"] in prev_lines:
                    info={}
                    info["Quantity"]=int(row["QUANTITY"])
                    info["Purchase price"]=float(row["PURCHASE PRICE"])
                    info["Selling price"]=float(row["SELLING PRICE"])
                    tmp[row["PRODUCT"]]=info

                else:
                    continue

            if quantity<=int(tmp[product_name]["Quantity"]):
                csv_writer.writerow({"PRODUCT":product_name,"QUANTITY":-quantity,"PURCHASE PRICE":list_lines[z]["PURCHASE PRICE"],"SELLING PRICE":list_lines[z]["SELLING PRICE"],"OPERATION":"sale","DATE TIME":datetime.strftime(datetime.now(), "%d-%m-%Y %H.%M.%S")})
                stop=1
            else:
                availability=int(tmp[product_name]["Quantity"])
                print(f"Max availability in the stock for the product {product_name}: {availability}")
                stop=0

            return stop

    stock_info()





    
  
def profit():

  """
  Function that prints the total gross and net profit 

  1-If operation type is "buy" the total price (price*quantity) is added in buying_cost variable
  2-If operation type is sale the total price (price*quantity) is added in gross_profit (keeping into account the sign)
  3-net profit is calculated as difference
  """

  with open("stock.csv") as csv_file:
        csv_reader=csv.DictReader(csv_file)
        list_lines=[]
        buying_cost=0
        gross_profit=0
        for row in list(csv_reader):
            list_lines.append(row)
        for line in list_lines:

            if line["OPERATION"]=="buy":
                buying_cost+=float(line["PURCHASE PRICE"])*int(line["QUANTITY"])
                
            if line["OPERATION"]=="sale":
                gross_profit+=-(float(line["SELLING PRICE"])*int(line["QUANTITY"]))
    
  net_profit=gross_profit-buying_cost
  print(f"Profit: gross = {gross_profit:.2f}€, net = {net_profit:.2f}€")
    







cmd = None
while cmd!="close":

    cmd = input("\nInsert a command: ")  
    
    if cmd=="sell":

        sold_dict={}
        answer="yes"
        
        while answer.lower()!="no":
            is_string,quantity=False,None

            while is_string==False:
                try:
                    product_name=input("Product name: ")  
                    if product_name.isnumeric():
                        is_string==False
                        raise ValueError("\nProduct not valid: inserted numeric value.")
                        continue
                    else:
                        is_string=True
                except ValueError as e:
                    print(e)

            while quantity==None:
                try:
                    quantity=int(input("Quantity: "))
                except ValueError:
                    print("\nQuantity not valid: insert integer number")
            
            #function sell is called: if stop=1 the sale is registered in a temporary dictionary sold_dict
            #and the user is asked if a new product has to be sold. If NO is selected the total selling price is shown
            stop=sell(product_name,quantity)
            stock=stock_info()
            
            if stop==1:
                sold_dict[product_name]=quantity
  
            answer=input("Do you want to insert another product?: [yes/NO]")
        
        if len(list(sold_dict.keys()))!=0:
            print("\nSALE SUCCESSFULLY REGISTERED: ")
            
        total_sold=0
        for key in list(sold_dict.keys()):
            print(f"{sold_dict[key]} X {key}: €{stock[key]['Selling price']}")
            total_sold+=(stock[key]['Selling price']*sold_dict[key])
        print(f"\nTotal: €{total_sold:.2f}")





    elif cmd=="profits":

        profit()


    
    elif cmd=="add":
    
        is_string,quantity=False,None
        while is_string==False:
            try:
                product_name=input("Product name: ")  
                if product_name.isnumeric():
                    is_string==False
                    raise ValueError("\nProdotto inserito non valido: inserito valore numerico.")
                    continue
                else:
                    is_string=True
            except ValueError as e:
                print(e)

        while quantity==None:
            try:
                quantity=int(input("Quantity: "))
            except ValueError:
                print("\nQuantity not valid: insert integer number")

        #function add is called
        add(product_name,quantity)





    elif cmd=="list":
    
        itemize()
        



    elif cmd=="help":
    
        help_cmd()




    
    elif cmd=="close":

        print("Bye Bye")
        break
    else:
        print("\nCommand not valid")
        help_cmd()